In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install datasets transformers accelerate


In [4]:
import torch
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import DatasetDict, Dataset

# Load train, validation, and test datasets
train_df = pd.read_csv("../input/civilsum-indian-legal-document-summarisation/train.csv")
valid_df = pd.read_csv("../input/civilsum-indian-legal-document-summarisation/valid.csv")
test_df = pd.read_csv("../input/civilsum-indian-legal-document-summarisation/test.csv")

# Keep only necessary columns
train_df = train_df[['text', 'summary']].dropna()
valid_df = valid_df[['text', 'summary']].dropna()
test_df = test_df[['text', 'summary']].dropna()


In [5]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

def tokenize_function(example):
    model_inputs = tokenizer(example["text"], max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(example["summary"], max_length=256, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Convert Pandas DataFrames to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text", "summary"])
valid_dataset = valid_dataset.map(tokenize_function, batched=True, remove_columns=["text", "summary"])
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=["text", "summary"])

# Create dataset dictionary
dataset = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset,
    "test": test_dataset
})


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Map:   0%|          | 0/21013 [00:00<?, ? examples/s]

Map:   0%|          | 0/1168 [00:00<?, ? examples/s]

Map:   0%|          | 0/1167 [00:00<?, ? examples/s]

In [6]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Adjust as per GPU memory
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=2,
    push_to_hub=False,
    logging_dir="./logs",
    logging_steps=50,
    fp16=torch.cuda.is_available(),  # Enable FP16 if using GPU
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)

trainer.train()


<ipython-input-9-7f7796cfc8a0>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

In [ ]:
model.save_pretrained("/kaggle/working/bart-legal-summary")
tokenizer.save_pretrained("/kaggle/working/bart-legal-summary")
